# A Text to Image Web App with Stable Diffusion, Fast API and React

Inspired by [Nicholas Renotte's YouTube video](https://www.youtube.com/watch?v=3l16wCsDglU), I'm coding up this as my weekend fun project. 

## Deploy the Model

To deploy the Stable Diffusion model, I took a shortcut to use SageMaker JumpStart: `Stable Diffusion 2.1 base` pretrained on [LAION-5B](https://laion.ai/blog/laion-5b/) is provided under SageMaker JumpStart's "Foundation Models: Image Generation" ML task. It can be deployed by one click, so that's what I did!

Find the model:

![SageMaker JumpStart](images/SageMaker-JumpStart.png)

One-click deploy it (here I used all the default settings for simplicity):
![Deploy](images/Model-deploy.png)

When the endpoint is ready in service, a sample notebook is provided.
![open notebook](images/Open-notebook.png)

By opening it in SageMaker Studio you can already play around with it. Since I have all my AWS credentials and Jupyter notebook server already set up locally, I downloaded it and had some fun in my local VSCode 😆. (You can find this notebook in my github repo.)

```python
# response = query_endpoint("cottage in impressionist style")
response = query_endpoint("a cat astronaut fighting aliens in space, realistic, high res")

img, prmpt = parse_response(response)

# Display hallucinated image
display_image(img,prmpt)
```

![](images/output-1.png)

## Build the API

### Fast and Minimum Start

I've wanted to try out FastAPI because it looks so fast and has a nice Swagger experience built in. So I first installed it as instructed in the [official website](https://fastapi.tiangolo.com/#installation):

```bash
# Install fastapi as well as the ASGI server Uvicorn
$ pip3 install fastapi
$ pip3 install uvicorn[standard]
```


Then I created a `main.py` file under my `api/` folder and typed in the minimal example of a FastAPI app:

```python
from typing import Union

from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}
```

Run the `uvicorn` dev server in my `/api` folder:

```bash
$ uvicorn main:app --reload
```

And as promised, I got a working API at `http://127.0.0.1L8000/` and my API doc Swagger UI at `http://127.0.0.1:8000/docs`, instantly 🚀!

![](images/fast-api.png)

![](images/swagger.png)

### Connect the API to the Model

Now it's time to make our API able to send a prompt to our Stable Diffusion model and get a response image. Let me first add a route and handler function for that:

```python
# api/main.py
@app.get("/generate-image")
def generate_image(prompt: str):
    image, prmpt = utils.parse_response(utils.query_endpoint(prompt))
    print(image)
    return {"out": "yeah"}
```

I picked the `query_endpoint()` function and the `parse_response()` function from the Stable Diffusion example Notebook that SageMaker JumpStart generated for me, and packed them in a `utils.py` file to be used in the API `main.py` file.

After saving the `main.py` file with the added new route, the Swagger UI conveniently added it and provides a test UI for me to input my prompt!

![](images/swagger-2.png)

And the model does send back the image generation! What does it look like? Well, it's an array of RGB channel values of each pixel in the image! Crazy, isn't it?

![](images/image-pixels.png)

### Process and Save the Image

To turn the magic numbers into image, I used [numpy](https://numpy.org/) and [PIL(Pillow)](https://pypi.org/project/Pillow/). For a quick test, I just added another utility function to save the image to disk (where the function is running).

```python
# api/utils.py
from PIL import Image
import numpy as np

# ...

def save_image(pixels):
    arr = np.array(pixels, dtype=np.uint8)
    img = Image.fromarray(arr)
    img.save("new.png")
```

Then in the API `/generate-image` route's handler, I plugged the image pixel array from response to this new utility function, and sent the prompt as the API response for now.

```python
# api/main.py
# ...
@app.get("/generate-image")
def generate_image(prompt: str):
    image, prmpt = utils.parse_response(utils.query_endpoint(prompt))
    utils.save_image(image)
    return {"prompt": prmpt}
```

Tested out in the Swagger UI with a new prompt: "A unicorn astronaut in space, full body from side".

![](images/swagger-3.png)

And in the `/api` folder a `new.png` appeared - the application works! Have a look at the unicorn astronaut image generated by Stable Diffusion and saved by the application:

![](images/new.png)

## Build the UI